## 测试gpt-generate

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch, os

# 使用你的 Hugging Face 令牌
HF_TOKEN = os.environ.get('HF_TOKEN')
if not HF_TOKEN:
    raise ValueError("Please set the HF_TOKEN environment variable")

# 加载模型和分词器
model_name = "openai-community/gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name, use_auth_token=HF_TOKEN)
model = GPT2LMHeadModel.from_pretrained(model_name, use_auth_token=HF_TOKEN)

# 生成文本
print("Generating text...", end="\n\n------------------\n\n")
input_text = "What's the capital of China."
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# 设置生成参数
max_length = 50
num_return_sequences = 1

# 生成文本
with torch.no_grad():
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=num_return_sequences)

# 解码生成的文本并按句子分行
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
sentences = generated_text.split('.')
formatted_text = '.\n\n'.join(sentence.strip() for sentence in sentences if sentence.strip())
print(formatted_text + '.')

## LORA微调: 第一层：幽默微调

### 📌 Step 1: 安装依赖项

In [2]:
%pip install -q transformers peft datasets accelerate torch tf-keras

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
import os

Note: you may need to restart the kernel to use updated packages.


d:\pycharm\myPrograms\pythonProject\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 📌 Step 2: 加载 GPT-2-Medium 和 Tokenizer

In [3]:
# 确保 CUDA 可用
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_name = "gpt2-medium"
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 测试 tokenizer 是否正常工作
text = "Hello, how are you today?"
tokens = tokenizer(text, return_tensors="pt").to(device)
print("Tokenized Input:", tokens)

Using device: cpu


d:\pycharm\myPrograms\pythonProject\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\HuggingFace\Transformers_CACHE\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Tokenized Input: {'input_ids': tensor([[15496,    11,   703,   389,   345,  1909,    30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


### 📌 Step 3: 加载数据集并进行 Tokenization

In [ ]:
dataset = load_dataset("text", data_files={"train": "humor.txt"})

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="longest", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

### 📌 Step 4: 配置 LoRA 并冻结 GPT-2 其他参数

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 自回归语言模型
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


### 📌 Step 5: 训练前检查

In [ ]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total trainable parameters: {count_trainable_parameters(model)}")

### 📌 Step 6: 训练 GPT-2-Medium + LoRA

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2_lora_output",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    fp16=True if torch.cuda.is_available() else False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()

### 📌 Step 7: 保存和测试微调后的模型

In [ ]:
output_dir = "./gpt2_lora_finetuned"
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# 生成测试示例
input_text = "Once upon a time"
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
output = model.generate(**input_ids, max_length=100)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)

### 📌 Step 8: 测试幽默效果

In [ ]:
from peft import PeftModel
from transformers import pipeline

# 加载 GPT-2 + 幽默 LoRA
model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
model = PeftModel.from_pretrained(model, "humor_lora")
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

# 生成幽默文本
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(text_gen("Once upon a time, a robot tried to tell a joke...", max_length=50))


### 📌 Step 9: 幽默指令微调

In [ ]:
# 1️⃣ 加载幽默 LoRA
model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
model = PeftModel.from_pretrained(model, "humor_lora")

# 2️⃣ 加载幽默指令数据集
dataset = load_dataset("json", data_files={"train": "funny_instructions.jsonl"})

def tokenize_function(examples):
    return tokenizer(examples["instruction"] + "\n" + examples["response"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 3️⃣ 配置 LoRA
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    r=16, lora_alpha=32, lora_dropout=0.1
)
model = get_peft_model(model, config)

# 4️⃣ 训练幽默指令 LoRA
training_args = TrainingArguments(
    output_dir="./funny_lora",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"]
)
trainer.train()

# 5️⃣ 保存幽默指令 LoRA
model.save_pretrained("funny_lora")


### 📌 合并幽默 LoRA + 指令 LoRA

In [ ]:
from peft import PeftModel

# 加载 GPT-2 并合并幽默 & 指令 LoRA
model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
model = PeftModel.from_pretrained(model, "funny_lora")

# 生成幽默回答
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(text_gen("Tell me a joke about AI.", max_length=50))
